In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend
from sklearn.metrics import classification_report, confusion_matrix

import shap
from operator import itemgetter

In [ ]:
os.getcwd()

In [ ]:
os.listdir('../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/')

In [ ]:
my_data_dir = '../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/'
train_path = my_data_dir + 'train/'
test_path = my_data_dir + 'test/'

image_shape = (300, 300, 1)
batch_size = 32 # according to your model and your choice

In [ ]:
print(image_shape[:2])

**Data Preparation**

In [ ]:
image_gen = ImageDataGenerator(rescale=1/255,
                              zoom_range=0.1,
                              brightness_range=[0.9, 1.0])
# rescale the image by normalizing it

In [ ]:
# we using keras inbuild function to ImageDataGenerator so we don't need to label 
# all images into 0 and 1 it automatically create it and batch also
# during training

# target_size = tuple of integers (height,width). the dimension
# to which all images found will be resized
# color_mode = whether the images will be converted to have 
# 1 or 3 channels 
# class_mode = mode for yielding th etargets. 'binary' : 1d numpy 
# array of binary labels
# batch_size = size of batches of data
train_set = image_gen.flow_from_directory(train_path, 
                                          target_size=image_shape[:2],
                                         color_mode='grayscale', 
                                          batch_size=batch_size,
                                         class_mode='binary',
                                          shuffle=True, seed=192)
test_set = image_gen.flow_from_directory(test_path,
                                        target_size=image_shape[:2],
                                        color_mode='grayscale',
                                        batch_size = batch_size,
                                        class_mode='binary',
                                        shuffle=False, seed=192)

In [ ]:
train_set.class_indices

In [ ]:
test_set.class_indices

In [ ]:
# convolutional model creation

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),
                 strides=2,
                 input_shape=image_shape,
                activation='relu',
                padding='valid'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3,3), 
                 strides=1, padding='valid',
                 input_shape=image_shape,
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=128, kernel_size=(3,3),
                padding='valid',
                input_shape=image_shape,
                activation='relu',
                strides=1))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Flatten())
model.add(Dense(224))
model.add(Activation('relu'))
model.add(Dropout(0.2))
# last layer
model.add(Dense(1))
model.add(Activation('sigmoid'))


model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
results = model.fit_generator(train_set, epochs=20, 
                              validation_data=test_set,
                             callbacks=[early_stop])

In [ ]:
plot_model(model, show_shapes=True, expand_nested=True, dpi=60)

In [ ]:
# Analyzing model performance
losses = pd.DataFrame(model.history.history)
losses[['loss', 'val_loss']].plot()

In [ ]:
losses[['accuracy', 'val_accuracy']].plot()

In [ ]:
# first we will find predict probability
pred_probability = model.predict_generator(test_set)

In [ ]:
# here it's true label for test set
test_set.classes

In [ ]:
# save the model
model.save('image_classification.h5')

In [ ]:
model_history = results.history
losses = pd.DataFrame(model_history)
#losses.index = map(lambda x : x+1, losses.index)
losses.head(3)

In [ ]:
pred_probability = model.predict_generator(test_set)
predictions = pred_probability > 0.5

plt.figure(figsize=(10,6))
plt.title('Confusion Matrix', size=20, weight='bold')
sns.heatmap(
     confusion_matrix(test_set.classes, predictions),
     annot=True,
     annot_kws={'size':14, 'weight': 'bold'},
     fmt='d',
     xticklabels=['Defect', 'OK'],
     yticklabels=['Defect', 'OK'])
plt.tick_params(axis='both', labelsize=14)
plt.ylabel('Actual', size=14, weight='bold')
plt.xlabel('Predicted', size=14, weight='bold')
plt.show()

In [ ]:
# defective is 0, ok is 1
print(classification_report(test_set.classes, predictions))

In [ ]:
from keras.models import load_model 

model= load_model('image_classification.h5')

In [ ]:
test_cases = ['ok_front/cast_ok_0_10.jpeg', 'ok_front/cast_ok_0_1026.jpeg', 'ok_front/cast_ok_0_1031.jpeg', 'ok_front/cast_ok_0_1121.jpeg', \
              'ok_front/cast_ok_0_1144.jpeg','def_front/cast_def_0_1059.jpeg', 'def_front/cast_def_0_108.jpeg', 'def_front/cast_def_0_1153.jpeg',\
              'def_front/cast_def_0_1238.jpeg', 'def_front/cast_def_0_1269.jpeg']
#test_cases = test_cases[:20]
plt.figure(figsize=(20,8))
for i in range(len(test_cases)):
    img_pred = cv2.imread(test_path + test_cases[i], cv2.IMREAD_GRAYSCALE)
    img_pred = img_pred / 255 # rescale
    prediction = model.predict(img_pred.reshape(1, *image_shape))
    
    img = cv2.imread(test_path + test_cases[i])
    label = test_cases[i].split("_")[0]
    
    plt.subplot(2, 5, i+1)
    plt.title(f"{test_cases[i].split('/')[1]}\n Actual Label : {label}", weight='bold', size=12)
    if (prediction < 0.5):
        predicted_label = "def"
        prob = (1-prediction.sum()) * 100
    else:
        predicted_label = "ok"
        prob = prediction.sum() * 100
        
    cv2.putText(img=img, text=f"Predicted Label : {predicted_label}", org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, color=(255, 0, 255), thickness=2)
    cv2.putText(img=img, text=f"Prediction : {'{:.3f}'.format(prediction.sum())}", org=(10, 280), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.7, color=(0, 255, 0), thickness=2)
    plt.imshow(img,cmap='gray')
    plt.axis('off')

plt.show()

In [ ]:
def predict_label(img):
    img_pred = img#cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    img_pred = img_pred/255
    img_pred = img_pred.reshape(-1, *image_shape)
    prediction = model.predict(img_pred)
    #print(prediction)
    if ((prediction[2]) == 0):
        predicted_label = "def"
        #prob = (1-prediction.sum()) * 100
    else:
        predicted_label = "ok"
        #prob = prediction.sum() * 100
        
    return predicted_label